# Microfossils, Remote Sensing, and GIS for Proxy-dating Coastal Archaeological Sites and Landscapes: A case from Minas Basin, Bay of Fundy, Canada

This Jupyter Notebook is intended to replicate the production of geochronological *terminus ante quem* boundaries for the Kingsport marsh as described in the associated article. 

Loading this *AppendixB* Jupyter Notebook as a Binder increases reproducability of the workflow employed to create the geochronological boundaries.

The notebook can be broken down as follows:

1. Import Packages


2. CSV Data Import and Jupyter Functionality


3. Correcting for Marine Resevoir Effect & Radiocarbon Calibration


4. Correlating Sea Level Proxy Samples to Oak Point High Water


5. Calculating Sea Level Rise (Higher High Water)


6. Calculating Equvalent Past HHW at Oak Point


7. Visualize Study Area


8. Proxy Dating Rooted Stumps in a Drowned Forest

## 1. Import Packages

The first step is to import the required Python packages to replicate the method presented in the associated article. 

Running the following codeblock imports the required packages. 

In [ ]:
import numpy as np
import pandas as pd
import iosacal
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
import os
import random
import requests
import datetime

from ipyleaflet import *
%matplotlib inline

from pprint import pprint
from numpy import ones,vstack
from numpy.linalg import lstsq
from iosacal import R, iplot
from iosacal.text import single_text
from ipywidgets import HTML
from pandas import *
from matplotlib import pyplot, figure
from IPython.display import clear_output
from random import choice
from ipywidgets import interact, interact_manual, fixed

## 2. CSV Data Import and Jupyter Functionality

Following the data preparation steps outlined in the methods section of the associated article, we have placed a CSV titled **Appendix A**  within the */data* folder. The following codeblock will display the *AppendixA.csv* file as a table.
<br><br>
The first line calls the CSV from the location within the project folder and loads it into a 'pandas' dataframe called *AppendixA*. Our second line defines formatting, cell alignment, and styles to be applied when we call the variable *Table*. *Table* is called in the third line and our formatted CSV is shown below.

In [ ]:
#read CSV file to pandas dataframe
AppendixA = pd.read_csv("./data/AppendixA.csv")

#create table style
Table = AppendixA.style.format({"error": lambda x: "±{:.1f}".format(abs(x))}, na_rep="-").format({"date": lambda x: "{:.1f} C14BP".format(float(x))}).format({"core_depth": lambda x: "{:.2f}m".format(abs(x))}, na_rep="-").format({"modern_HHW": lambda x: "{:.3f}m".format(float(x))}, na_rep="-").format({"HHW_corrected": lambda x: "{:.3f}m".format(float(x))}, na_rep="-").format({"modern_HHW": lambda x: "{:.3f}m".format(float(x))}, na_rep="-").format({"HHW_error": lambda x: "±{:.3f}m".format(float(x))}, na_rep="-").set_properties(subset=["error", "core_depth", "modern_HHW", "HHW_corrected", "HHW_error"], **{'text-align': 'center'}).set_properties(subset=["date"], **{'text-align': 'right'})

#show sylized table
Table

CSV data in the data frame can be quickly plotted by calling the attributes we wish to plot. The codeblock below plots the relationship of core depth to date of sample recovered from that depth in radiocarbon years.

In [ ]:
#load date and sample depth values from CSV as lists
date = AppendixA['date']
depth = AppendixA['core_depth']

#plot the list values as an x, y relationship
plt.plot(date, depth);

Once loaded into a data frame, records can be easily manipulated and displayed. This functionality is illustrated in the pertinent example that follows. 

#### Adjustment of error carried by Higher High Water values

The values that loaded to the `AppendixA` data frame include a vertical uncertainty in the column `HHW_error`. This uncertainty reflects the vertical extent of the marsh zone indicated by Foraminifera within each radiocarbon dated sample (Smith et al. 1984). It is pre-loaded to the field carrying errors associated with the analysis to illustrate the convenience of combining documentation and calculations in Jupyter Notebooks. 

Not reported in the table above are uncertainties associated with values in the `modern_HHW` column. The reported bulk accuracy of the CANEAST dataset from where these points were derived is 7.5cm (Robin et al. 2016). The same publication notes that inconsistencies appear greater in the Bay of Fundy (Robin et al. 2016, Fig.5), though the difference between tide station and modelled lower low water elevation shown in our southwestern Minas Basin study area is within the range of reported uncertainty. Therefore we can be reasonably confident that the values and uncertainty provided by the CANEAST dataset reflect true uncertainty of higher high water at the time of publication, 2016.

The following cell adds the vertical uncertainty of the `modern_HHW` points to the uncertainty of the foram derived sample elevations we will be using for this analysis. The following code performs the operation by overwriting the values in the initial data frame with the result, therefore eliminating excess variable creation. 

In [ ]:
#replace values in HHW error column with the sum of itself and 0.0075 metres
AppendixA["HHW_error"] = AppendixA["HHW_error"] + 0.075

#add AOCENOVNVIKDJFLOD
AppendixA["HHW_corrected"] = AppendixA["modern_HHW"]-AppendixA["core_depth"]

#show stylized table
Table

With the functionality of Jupyter Notebooks shown in the first step of this analysis. The next step is to prepare the radiocarbon dates for calibration.

## 3. Correcting for Marine Resevoir Effect & Radiocarbon Calibration

Now that we have our CSV holding the necessary data to begin, we need to correct for marine resevoir effect, then calibrate our radiocarbon dates. Mentioned in the associated article, the ΔR of -64 ±90 (obtained from [CALIB](url=http://calib.org/marine/)) is subtracted from each radiocarbon date before calibration occurs. The codeblock below will perform this calculation. As per usual, when addition or subtraction between two values with reported errors, the errors are summed.

In [ ]:
#apply marine resevoir correction to date and error values in table
AppendixA["date"] = AppendixA["date"] - (-64)
AppendixA["error"] = AppendixA["error"] + (90)

#show stylized table
Table

#### Calibrating sample data

The python package IOSACal is used in the following to calibrate the radiocarbon dates obtained from samples at Kingsport. 

The values in the associated article use the MARINE20 calibration curve released in 2020. The most recent IOSACal version (v0.4.1) does not come with the MARINE20 or INTCAL20 .14c calibration curve files, but were manually added to a local distribution for publication purposes. In the following, the exact same process is run on the same samples using the MARINE13 calibration curve file. Slightly different results ensue, but the code will be updated to reference the new calibration curve once IOSACal v0.5.0 is released.

Records from the CSV file are used as input for a loop calibrating the input dates in the following radiocarbon calibration. The output is a short report and a probability distribution plot for each of the calibrations.

To prepare for this process we create variables to hold the inputs and outputs of the calibration in the following codeblock. first three lines of code define the variables `s`, `d`, and `e` as values from specified columns in our dataframe (`ID`, `date`, & `error`). The following two lines initialize empty lists, `cal68` and `cal95`, that will store the minimum and maximum date of each confidence interval for each sample.

In [ ]:
#create variables for sample id, date, and date error
s = AppendixA.ID.values
d = AppendixA.date.values
e = AppendixA.error.values

#create a list to hold min and max values of probability distributions
cal68 = []
cal95 = []

A lot worthy of explaination happens in the codeblock.

Beginning with line 1, the following text described the process of our loop line by line.

Line 1 is the header of our `for` loop. The header defines the parameters of our loop, and describes the conditions to be met `for` the loop to stop. Our conditions state iterate the body of the loop below `for` a number of times equal to the `range` of `axis` `0`, or `rows`, in `AppendixA`. We have three samples, so the loop will iterate a total of three times with the provided data.<br>

Line 2 begins the body of the loop by defining a variable called `r` as equal to the values of date (`d`), error (`e`), and ID (`s`). The variables are designated as radiocarbon dates with the variable `R` (shorthand in IOSACal). The `i` in square brackets following `d`, `e`, and `s` is a reference to the value of `date`, `error`, and `ID` for the record with an index matching the current iteration of the loop. The `r` variable now holds a list containing the three values for each of the three samples. <br>

Line 3 passes the values of our list stored in `r` to the `calibrate` method in IOSACal. By prefixing `r` to `.calibrate` we are defining that the values in `r` will will be calibrated using the selected calibration curve. The calculated values are saved in the variable `cal_r`. <br>

Line 4 creates a report for the calibration, saved to the variable - `report`.<br>

On line 5, we search the output report text for digits, storing them in as a list in the variable called `findDigits`. Line 6 and 7 append the min and max values from our `findDigits` list for the 68% and 95% confidence intervals, based on their position in the list. These lines take advantage of the consistent structure IOSACal uses for reporting to allow us to directly use the outputs in further analysis.<br>

Lines 8 and 9 then `print` the report stored in our `report` variable below our code, and creates and saves a visualization of the calibration to the directory - `/SupplementaryData/output` as a JPG image named by `ID`.

**REMINDER:** *the most recent release of IOSACal (v.0.4.1) does not include the MARINE20 calibration curve. This notebook currently uses MARINE13 for radiocarbon calibration, while dates reported within the article use MARINE20. This results in slight variances between MARINE20 dates in the article and MARINE13 dates produced below. Upon release, IOSACal v0.5.0 will be automatically installed with the Binder image running this notebook, and code will promptly be updated to make use of the most recent dataset.*

In [ ]:
#define the output directory
dir= r"./output/"

#begin for loop iterating through all input records 
for i in range(AppendixA.shape[0]):
    
    #sort date, error, and sample id records and add input radiocarbon sample list
    r = R(d[i], e[i], s[i])
    
    #calibrate dates with the calibration curve shown in quotes
    cal_r = r.calibrate('marine13')
    
    #create text based output report for radiocarbon calibration
    report = (single_text(cal_r))
    
    #add all integer only words of one of more digits in the report to a list
    findDigits = [int(s) for s in re.findall(r'\b\d+\b', report)]
    
    #append the resulting list values to new lists
    #68% confidence interval values are location 7 and 8
    #95% confidence interval values are location 13 and 14
    cal68.append(findDigits[7:9])
    cal95.append(findDigits[13:15])
    
    #print report for current sample as output
    print(report)
    
    #show probability distribution plot as inline plot below output
    iplot(cal_r, output = dir + s[i] + ".jpg")

After performing radiocarbon calibration, the output lists are cast to a `numpy array` for storage as float values. These values are put into a `for` loop that iterates for `i` number of times. The number of iterations is set using the `range()` function with an input parameter of a list of values. For flexibility on the number of records that can be input to this code this value is equal the the number of records in our input table by calling the shape of any column with `AppendixA.shape[0]` (here we chose the first column `0`).

Within the `for` loop, simple calculations run over 1σ (68%) and 2σ (95%) confidence intervals to retrieve maximum, minimum, mean and associated uncertainty of each sample in calibrated years before present (CalBP) . These values are appended to the variables `cal95Date`,  `cal95Mean`,  `cal95Err`, `d`, and `e` for use later in the workflow.

In [ ]:
#create array objects to hold values not created above
cal68Date = np.array(cal68)
cal95Date = np.array(cal95)
cal95Mean = np.array(d)
cal95Err = np.array(e)

#begin for loop iterating through all input records 
for i in range(AppendixA.shape[0]):
    
    #calculate mean and uncertainty for 68% confidence intervals
    d[i] = (cal68Date[i,0]+cal68Date[i,1])/2
    e[i] = (cal68Date[i,0]-cal68Date[i,1])/2
    
    #calculate mean and uncertainty for 95% confidence intervals
    cal95Mean[i] = (cal95Date[i,0]+cal95Date[i,1])/2
    cal95Err[i] = (cal95Date[i,0]-cal95Date[i,1])/2

Results of the radiocarbon calibration are viewed in more detail below. The following codeblock creates and displays a new data frame formatted to show date range, mean, and uncertainty at both confidence intervals for all samples calibrated. The new data frame `calDates` 

In [ ]:
#define columns of radiocarbon calibration table
cols = ['2'+'\u03C3'+' Max Age', '2'+'\u03C3'+' Min Age']
cols2 = ['1'+'\u03C3'+' Max Age', '1'+'\u03C3'+' Min Age']
cols3 = ['2'+'\u03C3'+' Mean Age', '2'+'\u03C3'+' Uncertainty', '1'+'\u03C3'+' Mean Age', '1'+'\u03C3'+' Uncertainty'] 

#create an array holding mean date and uncertainty at each confidence interval
calMean = np.column_stack((cal95Mean,cal95Err,d,e))

#create variable to hold an int of the number of records
numRecs = len(AppendixA)

#load cal95 list to a data frame and add column titles
calDates = pd.DataFrame(cal95)
calDates.columns = cols

#add remaining datasets and column titles
calDates.reindex(columns=calDates.columns.tolist() + cols2)
calDates[cols2] = cal68
calDates.reindex(columns=calDates.columns.tolist() + cols3)
calDates[cols3] = calMean.reshape(numRecs,4)
calDates.insert(0, "Sample ID", AppendixA.iloc[0:numRecs,0])

#stylize and show full results of radiocarbon calibration in a table
calDates.style.hide_index().set_properties([], **{'text-align': 'center'}).format({'2'+'\u03C3'+' Max Age': lambda x: "{:.0f} CalBP".format(float(x))}).format({'2'+'\u03C3'+' Min Age': lambda x: "{:.0f} CalBP".format(float(x))}).format({'1'+'\u03C3'+' Mean Age': lambda x: "{:.0f} CalBP".format(float(x))}).format({'2'+'\u03C3'+' Mean Age': lambda x: "{:.0f} CalBP".format(float(x))}).format({'1'+'\u03C3'+' Max Age': lambda x: "{:.0f} CalBP".format(float(x))}).format({'1'+'\u03C3'+' Min Age': lambda x: "{:.0f} CalBP".format(float(x))}).format({'2'+'\u03C3'+' Uncertainty': lambda x: "±{:.0f} ".format(float(x))}).format({'1'+'\u03C3'+' Uncertainty': lambda x: "±{:.0f} ".format(float(x))}).set_properties(subset=['2'+'\u03C3'+' Uncertainty','1'+'\u03C3'+' Uncertainty'], **{'text-align': 'left'})

## *Section out of place for now*. Calculating Sea Level Rise (Higher High Water)

Our calculated HHW values for the Kingsport sample dates are shown above.

Next step is to determine the amount of RSL rise that has occurred since 1980 at the Kingsport sample locations. In order to do this, two coordinate pairs are required. Variables are defined below for our `x1`, `x2`, `y1`, and `y2` coordinates. Variable `x1` represents the radiocarbon equivalent date for the year 2016, where the value 0 is equal to the year 1950.

In [ ]:
#define the year today and year of hhw publication
yrToday = datetime.datetime.now().year
yrPub = 2016

#define modern date and errors to append to new data frame
confToday = [[1950-yrPub,0,1950-yrPub,0]]

#create list of hhw from elevations
hhw = pd.read_csv("./data/hhw.csv")
hhw = list(hhw.iloc[0:4,3])

#prepare data in data frame for processing
coordData = pd.DataFrame(calMean)
coordData = coordData.join(AppendixA)
coordData = coordData.drop(['ID', 'date', 'error','core_depth', 'modern_HHW'], axis=1)
coordData = coordData.append(confToday)
coordData = coordData.fillna(0)
coordData.iloc[3:4,4] = coordData.iloc[3:4,4] + float(hhw[3])
coordData.iloc[3:4,5] = coordData.iloc[3:4,5] + 0.2
coordData = coordData.reset_index(drop=True)
coordData

We use our coordinate pairs from our table to derive the equation for the linear relationship between the most recent of the radiocarbon dated samples (*Gx-6812, HHW_corrected*) and modern HHW for that sample (*Gx-6812, modern_HHW*) following a  slightly modified codeblock ["from Stack Overflow"](https://stackoverflow.com/a/21566184). This equation will be used to solve for the HHW in the year 1980 by solving for x = -30 in the two following codeblocks. 

The first codeblock creates an object called *coords* from our *x1, y1*, and *x2, y2* coordinate pairs of date and elevation. We then use <b>zip()</b> to sort these pairs into *x_coords* and *y_coords* lists. The third line creates the coefficient matrix (*A*) for the least squares method, <b>lstsq()</b>, on line 4. The two derived values are the slope (<i>m</i>) and intercept (<i>c</i>) of our equation, printed below.

In [ ]:
mList, bList, oldList, yngList = [],[],[],[]
iters = len(coordData.index)
i = 1
interval = 68

while i < iters:
    iters = len(coordData.index)
    #define x1 and x2 as sequential high water values
    if interval == 95:
        x1 = coordData.loc[iters-i].at[0]
        x2 = coordData.iloc[iters-i-1,0]
    else:
        x1 = coordData.loc[iters-i].at[2]
        x2 = coordData.loc[iters-i-1].at[2]
    y1 = coordData.loc[iters-i].at["HHW_corrected"]
    y2 = coordData.loc[iters-i-1].at["HHW_corrected"]
    
    #define a list of coordinate pairs
    coords = [(x1,y1),(x2,y2)]

    #re-arrange the lists then add to an array
    x_coords, y_coords = zip(*coords)
    A = vstack([x_coords,ones(len(x_coords))]).T

    #define variables m and b as the result of least-squares 
    #regression using the input coordinate pairs
    m, b = lstsq(A, y_coords, rcond=None)[0]
    yngList.append(x1)
    oldList.append(x2)
    mList.append(m)
    bList.append(b)

    #print the equation
    print("Equation for {:.0f} to {:.0f} CalBP is y = {:.5f}x + {:.5f}".format(x1,x2,m,b))
    i+=1

Our output equation is: $$y = -0.00230x + 6.63404$$<br>

The following two codeblocks perform two calculations. First, a modeled HHW elevation value is calaculated for Kingsport in 1980 by solving for <i>y</i>, where: $$x = -30$$<br>  Next, RSL rise since 1980 at HHW is calculated from the difference in HHW compared to 2020.<br>

The following codeblock can also be expressed as: $$HHW_{1980} = 0.00230(-30) + 6.63404$$<br>

Our sea level rise calculation in the second codeblock is expressed as: $$RSLR_{[1980,2020]} = HHW_{2020} - (0.00230(-30) + 6.6340)$$<br>



In [ ]:
yr=2021
inYr = (1950-yr)

SL = mList[0] * (inYr) + bList[0]

print("Higher high water was", "{:.3f}m".format(SL),"CGVD2013 at Oak Point in {:.0f}.".format(yrToday))

In [ ]:
hhwSmp = pd.read_csv("./data/hhw.csv", usecols=["ID"])
hhwZ = pd.read_csv("./data/hhw.csv", usecols=["hhw"])
hhw = hhwSmp.join(hhwZ)

#cast list to dictionary for input to widgets below
hhw = hhw.to_dict('split')

dd = widgets.Dropdown(
    options=hhw['data'],
    description='Modern HHW sample locations:',
    style={'description_width': 'initial'},
    value= hhw['data'][0][1]
)

text = widgets.Text(
    placeholder='select high water location',
    description='Reported elevation:',
    style={'description_width': 'initial'},
    value= str(hhw['data'][0][1]) + " m CGVD2013",
    disabled=False
)

sldMax = float()

def value_changed(change):
    sldMax = change.new
    text.value = str(change.new) + " m CGVD2013"
    
dd.observe(value_changed, 'value')

widgets.HBox([dd, text])

In [ ]:
#THIS CELL GIVES IMPRECISE VALUES FOR NOW

SLR = AppendixA.loc[2].at["modern_HHW"] - SL
SLRcm = SLR*100

print("Sea level has risen", "{:.1f}cm".format(SLRcm),"since 1980 at Oak Point.")

As our <i>HHW_corrected</i> elevations for the Kingsport core samples are calculated as values for the year 2020, we need to correct the elevations to match depth below 1980 HHW. The calculation is performed in the codeblock below and expressed as: $$HHW\_corrected_{AppendixA} = HHW\_corrected_{AppendixA} - SLR$$

In [ ]:
AppendixA["HHW_corrected"] = AppendixA["HHW_corrected"] - SLR
Table

## 4. Correlating Sea Level Proxy Samples to Oak Point High Water

#### Calculating Sample Higher High Water

##### REDO TEXT IN THIS SECTION. SHOULD ACTUALLY *ONLY* CREATE OAK POINT DATA FRAME WITH HHW VALUE FOR OAK POINT OFFSET BY DIFFERENCE BETWEEN CORE DEPTH AND HHW POINTS CLOSEST TO EACH SAMPLE. THIS OFFSET VALUE IS EQUAL TO THE DEPTH BELOW HIGHER HIGH WATER AT THE LOCATION THE CORE WAS TAKEN. THIS IS POSSIBLE BECAUSE BOTH HIGHER HIGH WATER VALUES ARE GRIDDED SAMPLE POINTS DERIVED FROM THE CANEAST DIGITAL OCEAN MODEL DATASET.

With radiocarbon calibration complete, production of the geochronological *terminus ante quem* boundaries can begin. Calculations shown below rely only on the 1σ (68%) confidence interval, while both intervals are used later to visualize the relative sea level over time.

First step is to calculate the elevation of the radiocarbon dated samples. These elevations will be used to derive past HHW in the CGVD2013 vertical datum.

The codeblock below subtracts *core_depth* from *modern_HHW* and places the result in the *HHW_corrected* column.

In [ ]:
#calculate past high water elevation as current high water subtract depth of sample
AppendixA["HHW_corrected"] = AppendixA["modern_HHW"]-AppendixA["core_depth"]

#restyle the Table dataframe to output CalBP
Table = AppendixA.style.hide_index().format({"error": lambda x: "±{:.1f}".format(abs(x))}, na_rep="-").format({"date": lambda x: "{:.0f} CalBP".format(float(x))}).format({"core_depth": lambda x: "{:.2f}m".format(abs(x))}, na_rep="-").format({"modern_HHW": lambda x: "{:.3f}m".format(float(x))}, na_rep="-").format({"HHW_corrected": lambda x: "{:.3f}m".format(float(x))}, na_rep="-").format({"modern_HHW": lambda x: "{:.3f}m".format(float(x))}, na_rep="-").format({"HHW_error": lambda x: "±{:.3f}m".format(float(x))}, na_rep="-").set_properties(subset=["error", "core_depth", "modern_HHW", "HHW_corrected", "HHW_error"], **{'text-align': 'center'}).set_properties(subset=["date"], **{'text-align': 'left'})

#show formatted table
Table

Sample `Gx-6811` was identified as representing marsh Zone 1b, in the lower portion of high marsh. These marsh zones were identified through surface sampling vegetation and Foraminifera along transects within the marsh (Smith et al, 1984). Marsh zone 1b was defined at Kingsport as having a vertical range of 55cm, and a maximum of 75cm below high tide. This means that at a minimum, the upper limit of Zone 1b will be 20cm below the limit of high water.

To adjust for the vertical position of sample `Gx-6811` in marsh Zone 1b, an offset of 20cm is added. This increases the elevation in the `HHW_corrected` field to represent the high water mark defining the limit of marsh Zone 1a. The codeblock below performs this correction, then displays the updated values in our `AppendixA` dataframe.

In [ ]:
#apply positive offset to account for the upper
#limit of zone 1b being 20cm below high-water
AppendixA.iloc[1:2,5] = AppendixA.iloc[1:2,5]+0.2

#show formatted table
Table

#### Calculating Equvalent Past HHW at Oak Point

An equivalent HHW elevation at Oak Point is obtained for each calibrated date by correcting for SLR and sample elevation.

An offset value for the geochronological boundary elevations at Oak Point is provided by the difference in RSL rise for the Kingsport sample because we can expect our high-marsh indicators to increase in elevation at the rate of RSL rise. 

The following codeblock calculates equivalent modern HHW elevations for Oak Point.

On line 1 the HHW value of `6.826` is used to calculate elevations for each sample subtracting <i>core_depth</i> and <i>SLR</i>. 

Line 2 displays the list of calculated values within <i>OakPointHHW</i>.

In [ ]:
OakPointHHW = [(6.826 - AppendixA["core_depth"] - SLR)-(6.826-AppendixA["modern_HHW"])]
OakPointHHW

The following codeblock creates a new data frame called `OakPoint`. This data frame relates the newly calculated HHW elevations for Oak Point with the calibrated dates, and errors from samples adjacent to the archaeological site.

To do this the new `OakPoint` data frame is created from a copy of our `AppendixA` data frame. The `AppendixA` HHW values are replaced with values calculated from the HHW taken from `OakPoint` data frame with values from the `OakPointHHW` list calculated above. 

Line 3 drops the unnecessary fields from the `AppendixA` data frame.

Lines 4 and 5 stylize out table, and display the result.

In [ ]:
#create OakPoint data frame from AppendixA data frame
OakPoint = pd.DataFrame(data=AppendixA)

#replace the HHW_corrected values with values from OakPointHHW - calculated above
OakPoint.iloc[0:3,5] = OakPointHHW[0]

#drop the ID, core_depth, and modern_HHW columns
OakPoint = OakPoint.drop(['ID','core_depth','modern_HHW'], axis=1)

#positive offset for foram. zone 1b adjusting to HHW value
OakPoint.iloc[1:2,2] = OakPoint.iloc[1:2,2]+0.2

#format the table
OakPointTable = OakPoint.style.hide_index().format({"error": lambda x: "±{:.1f}".format(abs(x))}, na_rep="-").format({"date": lambda x: "{:.1f} CalBP".format(float(x))}).format({"HHW_corrected": lambda x: "{:.3f}m".format(float(x))}, na_rep="-").format({"HHW_error": lambda x: "±{:.2f}m".format(float(x))}, na_rep="-").set_properties(subset=["error", "HHW_corrected", "HHW_error"], **{'text-align': 'center'}).set_properties(subset=["date"], **{'text-align': 'left'})

#show formatted table
OakPointTable

## 7. Visualize Study Area

An interactive map frame powered by iPyLeaflet is prepared by running the proceeding code block. The map frame is centered on the Oak Point study area at Kingsport marsh.

We define the map frame as *mapFrame* on line 1 followed by adding data and functionality: 

- ESRI satellite imagery basemap
- custom map tile layer from the lidar basemaps shown in the article
- measurement functionality
- full screen functionality
- scale bar
- layer toggle functionality

In [ ]:
#initialize map as "mapFrame"
mapFrame = Map(basemap=basemaps.Esri.WorldImagery, center=(45.1562107, -64.3699627), zoom=15, max_zoom=18)

#load and add local tile layer
basemap = LocalTileLayer(name="Shaded Relief",tms = True, attribution="Contains information licensed under the Open Government Licence – Nova Scotia. Created by: Wesley Weatherbee.", path="minasBase/{z}/{x}/{y}.png")
mapFrame.add_layer(basemap)

#load and add measurement function, fullscreen and layer toggle, and a visual scale
measure = MeasureControl(position='topleft',active_color = 'orange',primary_length_unit = 'meters')
mapFrame.add_control(measure)
measure.completed_color = 'red'
mapFrame.add_control(FullScreenControl(position='topright'))
mapFrame.add_control(LayersControl(position='topleft'))
mapFrame.add_control(ScaleControl(position='topright', metric=True))

##### Creation of the map data

The open map above displays datasets that will help explore the output boundaries after running this notebook. The extent of the shaded relief mirrors that of Figure 1 in the associated article presenting a high resolution maximum zoom level of 18 throughout.

The shaded relief layer was created from Lidar derived digital elevation models at 1m resolution from the [Province of Nova Scotia](url: "https://nsgi.novascotia.ca/datalocator/elevation/"). The layer is a series of compressed .png tiles within an organized folder directory using naming conventions relating to geolocation. This directory is located at */minasBase* off of the Binder root folder. 

The */minasBase* directory was automatically created from a Map Tile Package (.tpk) file exported from ArcGIS Pro. With the help of python modules `tpkutils` and `mbutil`, the .tpk was converted to a .mbtiles, MapBox tile package file then extracted to the */minasBase* directory. 

Optimization of the map tiles is responsible for the high zooom level display presented in this Binder. An original file count for the */minasBase* map tiles directory exceeded 5 million, which was not appreciated by GitHub or Binder. However, with the help of native python modules, the file count was reduced to approximately 37000 by removing any .png images in the directory that were a size of 190 bytes or smaller—files of this size contained no image data, only a transparency layer. 

#### Loading surveyed elevations of sea level proxy observations

Surveyed points have been gathered at Oak Point and throughout the mudflats of Minas Basin. Below we will load and visualize some survey data to the map frame. 

To differentiate the values a `Popup` method is used to visualize the `properties.Code` property of the points when a point is clicked. 


In [ ]:
#load geojson file containing surveyed but undated rsl indicator points
with open(r"./data/rslPoints.geojson", 'r') as f:
    data = json.load(f)

#reference geojson feature ID as variable 'features'
features = data['features']
indElev = []
zErr = []

#create layer group referencing json records within the map frame
rslGroup = LayerGroup(name="RSL Indicators", layers=())

#loop over geojson features
for i in range(len(features)):
    location=(features[i]['geometry']['coordinates'][1],features[i]['geometry']['coordinates'][0])
    indicators = features[i]['properties']['Code']
    indElev.append(features[i]['properties']['OHeight'])
    zErr.append(features[i]['properties']['zError'])
    ind = str(indicators)
    html = """<p><b>Indicator type: </b>""" + f"{ind}" + """</p>"""
    rslPoint = CircleMarker( radius=4, color="#000000", location=location, weight=1, opacity=1, 
                             fill_opacity=0.8, fill_color="#80bfff", draggable=False)
    
    # Popup associated to a layer
    rslPoint.popup = HTML(html)
    rslGroup.add_layer(rslPoint)

#add group layer created from combined rslPoint layers
mapFrame.add_layer(rslGroup)

#call and display the map frame
mapFrame   

## 8. Proxy Dating Rooted Stumps in Drowned Forest

The following were used to produce a plot of surveyed locations of stumps in the drowned forest in relation to sea level rise over-time. The cell below migrates calculated values from the *OakPoint* data frame in to a list. Current organization of the workflow required modern high water elevation and errors to be manually entered. Though this can be easily changed in the future.

In [ ]:
#load x values to plot from the OakPoint data frame  
#then append the modern date relative to 1950
x = list(coordData[2])

#load date uncertainty values to plot from the OakPoint 
#data frame  then append the modern date uncertainty
dateErr = list(coordData[3])
err98 = list(coordData[1])

#load y values to plot from the OakPoint data frame  
#then append the modern high water and uncertainty
y = list(coordData['HHW_corrected'])

error = list(coordData['HHW_error'])

Some placehoplder text for now.

In [ ]:
indAge = []
for i in range(len(features)):
    if indElev[i] >= y[2]:
        indAge.append((indElev[i]-bList[0])/mList[0])
    elif indElev[i] >= y[1]:
        indAge.append((indElev[i]-bList[1])/mList[1])
    elif indElev[i] >= y[0]:
        indAge.append((indElev[i]-bList[2])/mList[2])

Next, we create a plot using the same custom style used in the associated article. 

The cell below converts the lists above in to `numpy array` objects, obtains the values from the geojson indicator points we stored in variables previously, and plots the values using custom style parameters.

In [ ]:
#use the seaborn style colorblind in the plot
plt.style.use('seaborn-colorblind')

#define method to round values to the nearest 5
def myround(x, base=5):
    return int(base * round(float(x)/base))

#define x, y, errors, indicator elevation, z-error, calculated date, and date error
x = np.array(x)
dateErr = np.array(dateErr)
y = np.array(y)
error = np.array(error)
err98 = np.array(err98)
indAge = np.array(indAge)

#plot points, errorbars, interpolated rsl indicator age, and errorbar bounds
plt.plot(x, y, 'bo', x, y, 'k', linewidth=1.25, linestyle='dashed')
plt.plot(indAge, indElev, 'd', color='#4CE600', markersize=6, markeredgewidth=0.75, markeredgecolor='k')
plt.fill_betweenx(y, x-err98, x+err98, alpha=0.2, edgecolor='#A80000', facecolor='#A83800',
    linewidth=1.5, linestyle='dashdot', antialiased=True)
plt.fill_betweenx(y, x-dateErr, x+dateErr, alpha=0.4, edgecolor='#A80000', facecolor='#A83800',
    linewidth=1.5, linestyle='dashdot', antialiased=True)
plt.errorbar(x,y,error,dateErr, alpha=0.6, color='#1B2ACC', linewidth=0, elinewidth=1.5, ecolor='black', capsize=5)
plt.fill_between(x, y-error, y+error, alpha=0.75, edgecolor='#1B2ACC', facecolor='#51A8C7',
    linewidth=1.25, linestyle='dashdot', antialiased=True)

#stylize ticks, gridlines, bounds, and axis labels
plt.xticks(np.arange(0,5000, step=500))
plt.yticks(np.arange(-4,7.5, step=0.5))
plt.axhline(-4,color='black')
plt.axvline(-250,color='black')
plt.axhline(7.5,color='black')
plt.axvline(5000,color='black')
plt.tick_params(direction='in', length=6, width=1, colors='black',grid_color='gray', grid_alpha=0.5)
plt.xlabel('Age (years CalBP)')
plt.ylabel('Elevation (metres CGVD2013)')
plt.xlim(-250, 5000)
plt.ylim(-4, 7.5)

# Show the major grid lines with dark grey lines
plt.grid(b=True, which='major', color='#666666')

# Show the minor grid lines with very faint and almost transparent grey lines
plt.minorticks_on()
plt.grid(b=True, which='minor', color='#999999', alpha=0.2)

#plot the figure and set the size
fig = plt.gcf()
fig.set_size_inches(20, 12)
plt.savefig(r'./output/seaLevelCurve.png', edgecolor='black', dpi=400)

The result of our above plot shows the relative sea-level curve for Oak Point. The calibrated dates were produced with the calibration curve used in the above process, and elevations were corrected using HHW points from a CHS grid surface for higher high water. Uncertainties associated with each point are shown with vertical and horizontal bars.

A linear model is used to show interpolated uncertainty bounds over time. The light red boundary shows uncertainty of the radiocarbon calibration in the 95% confidence interval, while the 68% confidence interval is more opaque. Shown in blue boundary is uncertainty associated with the foram assemblage derived relative sea level indicator points.

There it is. Using the convenience of Jupyter Notebooks, powerful functionality of Python, and accessibility of MyBinder the Supplementary Data has interactively reproduced the analysis required to obtain a *terminus ante quem* proxy date from surveyed observations of coastal archaeological sites and landscapes.